# Import dependancies

In [1]:
import pandas as pd
from math import pi

In [2]:
!pip install plotly --upgrade --quiet
# Need latest version of plotly and need to pass renderers like this, for plotly to work in colab.
# Else you need to manually code a function, as plotly requires custom initialization.

In [3]:
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import plotly.graph_objects as go

# Insert the data for the observations, then display the same

In [4]:
V1 = [219, 219, 219, 219, 219, 218, 218, 218, 218]
I_fm = [0.843, 0.827, 0.810, 0.806, 0.796, 0.782, 0.763, 0.753, 0.731]
I_am = [3.78, 5.16, 6.93, 8.09, 10.17, 12.13, 15.14, 17.37, 18.82]
I_ag = [1.58, 3.02, 4.4, 6.03, 7.27, 8.98, 11.2, 13.05, 14.13]
I_fg = [1.19, 1.23, 1.26, 1.29, 1.34, 1.37, 1.43, 1.47, 1.45]

# Check if you have entered all the observations for all the required quantities
print(len(V1))
print(len(I_fm))
print(len(I_am))
print(len(I_ag))
print(len(I_fg))

# Resistance of motor and generator in ohms
R_am = 0.8
R_ag = 1.4

n = len(V1)
S_No = list(range(1, n + 1))

df_dict = {
    'S.No.': S_No ,
    'V1 (V)': V1 ,
    'I_fm (A)': I_fm ,
    'I_am (A)': I_am ,
    'I_ag (A)': I_ag ,
    'I_fg (A)': I_fg ,
}

df = pd.DataFrame(df_dict)
df.set_index('S.No.', inplace=True)

display(df.T)

9
9
9
9
9


S.No.,1,2,3,4,5,6,7,8,9
V1 (V),219.000,219.000,219.00,219.000,219.000,218.000,218.000,218.000,218.000
I_fm (A),0.843,0.827,0.81,0.806,0.796,0.782,0.763,0.753,0.731
I_am (A),3.780,5.160,6.93,8.090,10.170,12.130,15.140,17.370,18.820
I_ag (A),1.580,3.020,4.40,6.030,7.270,8.980,11.200,13.050,14.130
I_fg (A),1.190,1.230,1.26,1.290,1.340,1.370,1.430,1.470,1.450


# Perform calculations and display table for calculation of stray losses

In [5]:
V = 219

df1 = df.copy()
df1.drop(['V1 (V)', 'I_fm (A)', 'I_am (A)', 'I_ag (A)', 'I_fg (A)'], axis=1, inplace=True)

df1['I_a (A)'] = df['I_am (A)'] - df['I_ag (A)']
df1['I_am².R_am'] = R_am * (df['I_am (A)'] ** 2)
df1['I_ag².R_ag'] = R_ag * (df['I_ag (A)'] ** 2)
df1['P_in (arm)'] = V * df1['I_a (A)']
df1['P_a'] = df1['I_am².R_am'] + df1['I_ag².R_ag']
df1['P_in (arm) - P_a'] = df1['P_in (arm)'] - df1['P_a']
df1['P_s'] = 1/2 * df1['P_in (arm) - P_a'] # Stray Loss for each machine

display(df1.T)

S.No.,1,2,3,4,5,6,7,8,9
I_a (A),2.20000,2.14000,2.53000,2.06000,2.90000,3.15000,3.94000,4.32000,4.69000
I_am².R_am,11.43072,21.30048,38.41992,52.35848,82.74312,117.70952,183.37568,241.37352,283.35392
I_ag².R_ag,3.49496,12.76856,27.10400,50.90526,73.99406,112.89656,175.61600,238.42350,279.51966
P_in (arm),481.80000,468.66000,554.07000,451.14000,635.10000,689.85000,862.86000,946.08000,1027.11000
P_a,14.92568,34.06904,65.52392,103.26374,156.73718,230.60608,358.99168,479.79702,562.87358
P_in (arm) - P_a,466.87432,434.59096,488.54608,347.87626,478.36282,459.24392,503.86832,466.28298,464.23642
P_s,233.43716,217.29548,244.27304,173.93813,239.18141,229.62196,251.93416,233.14149,232.11821


# Perform calculations and display table for calculation of efficiency

In [6]:
df2 = df.copy()
df2.drop(['V1 (V)', 'I_fm (A)', 'I_am (A)', 'I_ag (A)', 'I_fg (A)'], axis=1, inplace=True)

df2['Motor Input'] = V * (df['I_fm (A)'] + df['I_am (A)'])
df2['Motor Armature Copper Loss'] = R_am * df['I_am (A)'] ** 2
df2['Motor Field Copper Loss'] = V * df['I_fm (A)']
df2['Motor Total Loss'] = df1['P_s'] + df2['Motor Armature Copper Loss'] + df2['Motor Field Copper Loss']
df2['Motor Output'] = df2['Motor Input'] - df2['Motor Total Loss']
df2['% η_m'] = df2['Motor Output'] / df2['Motor Input'] * 100

df2['Generator Output'] = V * (df['I_ag (A)'] - df['I_fg (A)'])
df2['Generator Armature Copper Loss'] = (df['I_ag (A)'] ** 2) * R_ag
df2['Generator Field Copper Loss'] = V * df['I_fg (A)']
df2['Generator Total Loss'] = df1['P_s'] + df2['Generator Armature Copper Loss'] + df2['Generator Field Copper Loss']
df2['Generator Input'] = df2['Generator Output'] + df2['Generator Total Loss']
df2['% η_g'] = df2['Generator Output'] / df2['Generator Input'] * 100

display(df2.T)

S.No.,1,2,3,4,5,6,7,8,9
Motor Input,1012.437000,1311.153000,1695.060000,1948.224000,2401.554000,2827.728000,3482.757000,3968.937000,4281.669000
Motor Armature Copper Loss,11.430720,21.300480,38.419920,52.358480,82.743120,117.709520,183.375680,241.373520,283.353920
Motor Field Copper Loss,184.617000,181.113000,177.390000,176.514000,174.324000,171.258000,167.097000,164.907000,160.089000
Motor Total Loss,429.484880,419.708960,460.082960,402.810610,496.248530,518.589480,602.406840,639.422010,675.561130
Motor Output,582.952120,891.444040,1234.977040,1545.413390,1905.305470,2309.138520,2880.350160,3329.514990,3606.107870
% η_m,57.579101,67.989322,72.857423,79.324215,79.336358,81.660560,82.703162,83.889338,84.222014
Generator Output,85.410000,392.010000,687.660000,1038.060000,1298.670000,1666.590000,2139.630000,2536.020000,2776.920000
Generator Armature Copper Loss,3.494960,12.768560,27.104000,50.905260,73.994060,112.896560,175.616000,238.423500,279.519660
Generator Field Copper Loss,260.610000,269.370000,275.940000,282.510000,293.460000,300.030000,313.170000,321.930000,317.550000
Generator Total Loss,497.542120,499.434040,547.317040,507.353390,606.635470,642.548520,740.720160,793.494990,829.187870


# Show required plots

In [7]:
import math

# utility function for the plotting function below. il stands for interval length, ul stands for upper limit.
def get_il_and_ul(series, num_intervals=10):
  # il = ( int(series.max()) + il ) / num_intervals
  il = math.ceil( int(series.max()) / (num_intervals - 1) )
  ul = int(series.max()) + il
  return ul, il

def show_plot(y_series, x_series):
  x_ul, x_il = get_il_and_ul(x_series)
  y_ul, y_il = get_il_and_ul(y_series)
  x_tickvals_list = list(range(0, x_ul, x_il))
  y_tickvals_list = list(range(0, y_ul, y_il))

  x_label = x_series.name
  y_label = y_series.name
  scatter = go.Scatter(x=x_series, y=y_series, mode='markers', name=f"{y_label} vs {x_label}")

  layout = go.Layout(
      title={'text': f"{y_label} vs {x_label}", 'x': 0.5, 'y': 0.95},
      xaxis=dict( title=f"{x_label}", titlefont_size=14, tickfont_size=12, range=[0, x_ul], tickvals=x_tickvals_list ),
      yaxis=dict( title=f"{y_label}", titlefont_size=14, tickfont_size=12, range=[0, y_ul], tickvals=y_tickvals_list ),
      autosize=False,
      width=600, height=400, margin=dict(l=100, r=50, b=100, t=100, pad=5),
      paper_bgcolor='LightSteelBlue',
      xaxis_showgrid=True,
      yaxis_showgrid=True,
  )

  fig = go.Figure(data=[scatter], layout=layout)

  fig.show(renderer="colab")

## Plot efficiency of motor vs line current

In [8]:
df['I_am + I_fm'] = df['I_am (A)'] + df['I_fm (A)']
show_plot(df2['% η_m'], df['I_am + I_fm'])

## Plot efficiency of motor vs motor output

In [9]:
show_plot(df2['% η_m'], df2['Motor Output'])

## Plot efficiency of generator vs load current of generator

In [10]:
show_plot(df2['% η_g'], df['I_ag (A)'])

## Plot efficiency of generator vs generator output

In [11]:
show_plot(df2['% η_g'], df2['Generator Output'])